In [1]:
import io
import os
import sys
import time
import tempfile
import platform
import subprocess
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Loading the api keys.
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')
google_key = os.getenv('GOOGLE_API_KEY')

if openai_key and google_key:
    print('All good🤙')
else:
    print('Check your api keys')

All good🤙


In [3]:
# Initializing our clients
openai = OpenAI()
googleai = OpenAI(api_key=google_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

# Our models
OPENAI_MODEL = "gpt-4o-mini"
GOOGLEAI_MODEL = "gemini-2.5-flash-preview-04-17"

In [4]:
# Determinig which machine the user is running.
def get_machine_info():
    arch = platform.machine().lower()
    system = platform.system().lower()
    
    # Normalize architecture names
    if arch in ['amd64', 'x86_64']:
        arch = 'x86_64'
    elif arch in ['aarch64', 'arm64']:
        arch = 'arm64'
    
    return f"{arch}-{system}"

machine = get_machine_info()
print(machine)

x86_64-windows


In [5]:
def create_system_prompt(source_lang, target_lang, machine=machine):
    system_prompt = f"You are an expert at transpiling {source_lang} code to highly optimized {target_lang} for {machine}. "
    system_prompt += "Supported target languages: C++, Rust, Go, Java, Assembly"
    system_prompt += f"Respond only with {target_lang} code. Use minimal comments. "
    system_prompt += f"Focus on producing identical output with maximum performance. "
    
    # Add language-specific rules for source languages
    if source_lang.lower() in ['python', 'javascript']:
        system_prompt += "Handle dynamic types by inferring static types from usage. "
        system_prompt += "Replace garbage collection with stack allocation or smart pointers. "
    
    # Add language-specific rules for target languages
    if target_lang.lower() in ['cpp', 'c++']:
        system_prompt += "Use modern C++ features (C++17+). Include all necessary headers. "
        system_prompt += "Avoid undefined behavior and buffer overflows. "
    elif target_lang.lower() == 'c':
        system_prompt += "Use C99 or C11 standard. Include all necessary headers. "
        system_prompt += "Manage memory manually with malloc/free. "
    elif target_lang.lower() == 'rust':
        system_prompt += "Follow Rust ownership rules. Use appropriate lifetimes. "
    elif target_lang.lower() in ['assembly', 'asm']:
        system_prompt += f"Target {machine} assembly. Use efficient register allocation. "
    elif target_lang.lower() == 'go':
        system_prompt += "Use Go idioms. Handle errors explicitly. "
    elif target_lang.lower() == 'java':
        system_prompt += "Use modern Java features. Handle exceptions properly. "
    
    return system_prompt

In [6]:
def create_user_prompt(source_lang, target_lang, code):
    user_prompt = f"Transpile this {source_lang} code to optimized {target_lang}: \n\n"
    user_prompt += "Supported target languages: C++, Rust, Go, Java, Assembly"
    user_prompt += f"Requirements:\n"
    user_prompt += f"- Identical output behavior\n"
    user_prompt += f"- Maximum performance\n"
    user_prompt += f"- Handle edge cases (overflows, bounds, null checks)\n"
    user_prompt += f"- Only respond with {target_lang} code\n\n"
    user_prompt += f"{source_lang} Code:\n{code}"
    
    return user_prompt

In [7]:
def create_messages(source_lang, target_lang, code, machine=machine):
    return [
        {"role": "system", "content": create_system_prompt(source_lang, target_lang, machine)},
        {"role": "user", "content": create_user_prompt(source_lang, target_lang, code)}
    ]

In [8]:
def write_output(code, target_lang, filename="optimized"):
    # Map languages to file extensions
    extensions = {
        'cpp': '.cpp',
        'c++': '.cpp',
        'rust': '.rs',
        'c': '.c',
        'assembly': '.asm',
        'asm': '.asm',
        'python': '.py',
        'javascript': '.js',
        'java': '.java',
        'go': '.go'
    }
    
    # Get extension or default to .txt
    ext = extensions.get(target_lang.lower(), '.txt')
    
    # Clean code (remove markdown code blocks)
    cleaned_code = code.replace(f"```{target_lang.lower()}", "").replace("```", "")
    
    # Write to file
    output_file = f"{filename}{ext}"
    with open(output_file, "w") as f:
        f.write(cleaned_code)
    
    return output_file  # Return filename for confirmation

In [9]:
def optimize_gpt(source_lang, target_lang, code, machine=machine):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=create_messages(source_lang, target_lang, code, machine), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    output_file = write_output(reply, target_lang)
    print(f"\nCode saved to: {output_file}")

In [10]:
def optimize_gemini(source_lang, target_lang, code, machine=machine):    
    stream = googleai.chat.completions.create(model=GOOGLEAI_MODEL, messages=create_messages(source_lang, target_lang, code, machine), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    output_file = write_output(reply, target_lang)
    print(f"\nCode saved to: {output_file}")

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

In [ ]:
optimize_gpt(source_lang='python', target_lang='cpp', code=pi)

In [ ]:
# Executing the python code
exec(pi)

In [ ]:
# Compiling and running the executable.
!g++ -o optimized optimized.cpp
!optimized.exe

In [ ]:
optimize_gemini(source_lang='python', target_lang='cpp', code=pi)

In [ ]:
!g++ -o gemini_optimized optimized.cpp
!gemini_optimized.exe

In [12]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
optimize_gpt(source_lang='python', target_lang='cpp', code=python_hard)

In [ ]:
# Compiling and running the executable.
!g++ -o optimized optimized.cpp
!optimized.exe

In [ ]:
optimize_gemini(source_lang='python', target_lang='cpp', code=python_hard)

In [ ]:
# Compiling and running the executable.
!g++ -o optimized optimized.cpp
!optimized.exe

In [33]:
def stream_gpt(source_lang, target_lang, code, machine=machine):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=create_messages(source_lang, target_lang, code, machine), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [34]:
def stream_gemini(source_lang, target_lang, code, machine=machine):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=create_messages(source_lang, target_lang, code, machine), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [13]:
def optimize(model, source_lang, target_lang, code, machine=machine):
    if model=="GPT":
        result = stream_gpt(source_lang, target_lang, code, machine)
    elif model=="Gemini":
        result = stream_gemini(source_lang, target_lang, code, machine)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        source_code = gr.Textbox(label="Source Code:", lines=10, value=pi)
        target_code = gr.Textbox(label="Target Code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
        source_lang = gr.Dropdown(["Python", "Javascript"], label="Select Source", value="Python")
        target_lang = gr.Dropdown(["C++", "Rust", "Assembly", "Java", "Go"], label="Select Target")
        machine = gr.Textbox(label="Machine", interactive=False)
    with gr.Row():
        convert = gr.Button("Convert code")


        # Automatically populate machine_info on load
        ui.load(get_machine_info, inputs=[], outputs=[machine])
    
    convert.click(optimize, inputs=[model, source_lang, target_lang, source_code, machine], outputs=[target_code])

ui.launch(inbrowser=True)

In [14]:
def write_temp_file(code, extension):
    """Strip language tag and write code to a temporary file. Return file path."""

    # Clean code: remove first line if it looks like a language tag
    code = code.strip()
    lines = code.splitlines()
    language_tags = {"rust", "cpp", "c++", "c", "java", "go", "asm", "assembly", "python"}

    if lines and lines[0].strip().lower() in language_tags:
        code = "\n".join(lines[1:])

    # Write to temp file
    with tempfile.NamedTemporaryFile(mode='w', suffix=extension, delete=False) as f:
        f.write(code)
        return f.name


In [15]:
def execute_python(code):
    """Execute Python code and return output with runtime"""
    try:
        start_time = time.time()
        output = io.StringIO()
        sys.stdout = output
        exec(code)
        end_time = time.time()
        result = output.getvalue()
        runtime = f"Runtime: {(end_time - start_time):.4f} seconds"
        return f"{result}\n{runtime}"
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        sys.stdout = sys.__stdout__

In [16]:
def execute_javascript(code):
    """Execute JavaScript code using Node.js"""
    try:
        temp_file = write_temp_file(code, '.js')
        start_time = time.time()
        result = subprocess.run(['node', temp_file], capture_output=True, text=True, timeout=30)
        end_time = time.time()
        runtime = f"Runtime: {(end_time - start_time):.4f} seconds"
        
        if result.returncode == 0:
            return f"{result.stdout}\n{runtime}"
        else:
            return f"Error: {result.stderr}"
    except subprocess.TimeoutExpired:
        return "Error: Code execution timed out"
    except FileNotFoundError:
        return "Error: Node.js not found. Please install Node.js to run JavaScript code."
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        if 'temp_file' in locals():
            os.unlink(temp_file)

In [17]:
def execute_cpp(code):
    """Compile and execute C++ code"""
    try:
        temp_file = write_temp_file(code, '.cpp')
        executable = temp_file.replace('.cpp', '')
        
        # Compile
        compile_cmd = ["g++", "-std=c++17", "-O2", "-o", executable, temp_file]
        compile_result = subprocess.run(compile_cmd, capture_output=True, text=True, timeout=30)
        
        if compile_result.returncode != 0:
            return f"Compilation Error: {compile_result.stderr}"
        
        # Execute
        start_time = time.time()
        run_result = subprocess.run([executable], capture_output=True, text=True, timeout=30)
        end_time = time.time()
        runtime = f"Runtime: {(end_time - start_time):.4f} seconds"
        
        if run_result.returncode == 0:
            return f"{run_result.stdout}\n{runtime}"
        else:
            return f"Runtime Error: {run_result.stderr}"
            
    except subprocess.TimeoutExpired:
        return "Error: Code execution timed out"
    except FileNotFoundError:
        return "Error: g++ compiler not found. Please install g++ to compile C++ code."
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        # Clean up temporary files
        if 'temp_file' in locals():
            try:
                os.unlink(temp_file)
                if 'executable' in locals() and os.path.exists(executable):
                    os.unlink(executable)
            except:
                pass

In [18]:
def execute_rust(code):
    """Compile and execute Rust code"""
    try:
        temp_file = write_temp_file(code, '.rs')
        executable = temp_file.replace('.rs', '')
        
        # Compile
        compile_cmd = ["rustc", "-O", "-o", executable, temp_file]
        compile_result = subprocess.run(compile_cmd, capture_output=True, text=True, timeout=30)
        
        if compile_result.returncode != 0:
            return f"Compilation Error: {compile_result.stderr}"
        
        # Execute
        start_time = time.time()
        run_result = subprocess.run([executable], capture_output=True, text=True, timeout=30)
        end_time = time.time()
        runtime = f"Runtime: {(end_time - start_time):.4f} seconds"
        
        if run_result.returncode == 0:
            return f"{run_result.stdout}\n{runtime}"
        else:
            return f"Runtime Error: {run_result.stderr}"
            
    except subprocess.TimeoutExpired:
        return "Error: Code execution timed out"
    except FileNotFoundError:
        return "Error: rustc compiler not found. Please install Rust to compile Rust code."
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        if 'temp_file' in locals():
            try:
                os.unlink(temp_file)
                if 'executable' in locals() and os.path.exists(executable):
                    os.unlink(executable)
            except:
                pass

In [19]:
def execute_go(code):
    """Compile and execute Go code"""
    try:
        temp_file = write_temp_file(code, '.go')
        
        # Go run compiles and executes in one step
        start_time = time.time()
        result = subprocess.run(['go', 'run', temp_file], capture_output=True, text=True, timeout=30)
        end_time = time.time()
        runtime = f"Runtime: {(end_time - start_time):.4f} seconds"
        
        if result.returncode == 0:
            return f"{result.stdout}\n{runtime}"
        else:
            return f"Error: {result.stderr}"
            
    except subprocess.TimeoutExpired:
        return "Error: Code execution timed out"
    except FileNotFoundError:
        return "Error: Go compiler not found. Please install Go to run Go code."
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        if 'temp_file' in locals():
            os.unlink(temp_file)

In [20]:
def execute_java(code):
    """Compile and execute Java code"""
    try:
        # Extract class name from code
        import re
        class_match = re.search(r'public\s+class\s+(\w+)', code)
        if not class_match:
            return "Error: Could not find public class declaration"
        
        class_name = class_match.group(1)
        temp_file = write_temp_file(code, '.java')
        temp_dir = os.path.dirname(temp_file)
        
        # Rename file to match class name
        java_file = os.path.join(temp_dir, f"{class_name}.java")
        os.rename(temp_file, java_file)
        
        # Compile
        compile_cmd = ["javac", java_file]
        compile_result = subprocess.run(compile_cmd, capture_output=True, text=True, timeout=30)
        
        if compile_result.returncode != 0:
            return f"Compilation Error: {compile_result.stderr}"
        
        # Execute
        start_time = time.time()
        run_result = subprocess.run(['java', '-cp', temp_dir, class_name], 
                                  capture_output=True, text=True, timeout=30)
        end_time = time.time()
        runtime = f"Runtime: {(end_time - start_time):.4f} seconds"
        
        if run_result.returncode == 0:
            return f"{run_result.stdout}\n{runtime}"
        else:
            return f"Runtime Error: {run_result.stderr}"
            
    except subprocess.TimeoutExpired:
        return "Error: Code execution timed out"
    except FileNotFoundError:
        return "Error: Java compiler not found. Please install Java JDK to compile Java code."
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        if 'java_file' in locals():
            try:
                os.unlink(java_file)
                class_file = java_file.replace('.java', '.class')
                if os.path.exists(class_file):
                    os.unlink(class_file)
            except:
                pass

In [21]:

def execute_assembly(code):
    """Assemble and execute Assembly code (x86-64 NASM)"""
    try:
        temp_file = write_temp_file(code, '.asm')
        object_file = temp_file.replace('.asm', '.o')
        executable = temp_file.replace('.asm', '')
        
        # Assemble
        assemble_cmd = ["nasm", "-f", "elf64", "-o", object_file, temp_file]
        assemble_result = subprocess.run(assemble_cmd, capture_output=True, text=True, timeout=30)
        
        if assemble_result.returncode != 0:
            return f"Assembly Error: {assemble_result.stderr}"
        
        # Link
        link_cmd = ["ld", "-o", executable, object_file]
        link_result = subprocess.run(link_cmd, capture_output=True, text=True, timeout=30)
        
        if link_result.returncode != 0:
            return f"Linking Error: {link_result.stderr}"
        
        # Execute
        start_time = time.time()
        run_result = subprocess.run([executable], capture_output=True, text=True, timeout=30)
        end_time = time.time()
        runtime = f"Runtime: {(end_time - start_time):.4f} seconds"
        
        if run_result.returncode == 0:
            return f"{run_result.stdout}\n{runtime}"
        else:
            return f"Runtime Error: Exit code {run_result.returncode}"
            
    except subprocess.TimeoutExpired:
        return "Error: Code execution timed out"
    except FileNotFoundError:
        return "Error: NASM assembler or linker not found. Please install NASM and binutils."
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        for file in [temp_file, object_file, executable]:
            if 'file' in locals() and os.path.exists(file):
                try:
                    os.unlink(file)
                except:
                    pass

In [22]:
def execute_source_code(source_lang, code):
    """Execute code in the source language"""
    if source_lang == "Python":
        return execute_python(code)
    elif source_lang == "Javascript":
        return execute_javascript(code)
    else:
        return f"Error: Unsupported source language: {source_lang}"

In [23]:
def execute_target_code(target_lang, code):
    """Execute code in the target language"""
    if target_lang == "C++":
        return execute_cpp(code)
    elif target_lang == "Rust":
        return execute_rust(code)
    elif target_lang == "Go":
        return execute_go(code)
    elif target_lang == "Java":
        return execute_java(code)
    elif target_lang == "Assembly":
        return execute_assembly(code)
    else:
        return f"Error: Unsupported target language: {target_lang}"

In [24]:
def clean_code_block(code):
    """Remove markdown-style language tags like 'rust', 'java', etc. from the first line of pasted code."""
    language_tags = {"rust", "cpp", "c++", "c", "java", "go", "asm", "assembly", "python"}
    
    lines = code.strip().splitlines()
    if lines and lines[0].strip().lower() in language_tags:
        return "\n".join(lines[1:])
    return code

In [25]:
css = """
/* Base transparent classes */
.blue, .black {
  background-color: transparent;
}

/* Light Blue Glow */
.glass-card {
  width: 100%;
  padding: 1rem;
  border-radius: 16px;
  background: rgba(173, 216, 230, 0.2);
  box-shadow: 0 0 12px rgba(173, 216, 230, 0.6);
  backdrop-filter: blur(4px);
  -webkit-backdrop-filter: blur(4px);
  border: 1px solid rgba(173, 216, 230, 0.4);
  color: white;
  position: relative;
  overflow: hidden;
}

/* Black Glow */
.black-glow-card {
  width: 100%;
  padding: 1rem;
  border-radius: 16px;
  background: rgba(0, 0, 0, 0.2);
  box-shadow: 0 0 14px rgba(0, 0, 0, 0.8);
  backdrop-filter: blur(4px);
  -webkit-backdrop-filter: blur(4px);
  border: 1px solid rgba(0, 0, 0, 0.5);
  color: white;
  position: relative;
  overflow: hidden;
}

/* Animated Border Edge */
.edge-animate::before {
  content: "";
  position: absolute;
  top: 0;
  left: -50%;
  width: 200%;
  height: 3px;
  background: linear-gradient(90deg, transparent, #00f6ff, transparent);
  animation: moveEdge 3s linear infinite;
}

/* Keyframes */
@keyframes moveEdge {
  0% { left: -50%; }
  100% { left: 100%; }
}
"""


In [35]:
with gr.Blocks(title=" 🧶LangWeaver", css=css,) as ui:
    with gr.Row(elem_classes=["blur", "glass-card","edge-animate"]):
        source_code = gr.Textbox(label="Source Code:", lines=20, value=pi, elem_classes=["black", "black-glow-card"])
        target_code = gr.Textbox(label="Target Code:", lines=20, elem_classes=["black", "black-glow-card"])
    with gr.Row():
        model = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
        source_lang = gr.Dropdown(["Python", "Javascript"], label="Select Source", value="Python")
        target_lang = gr.Dropdown(["C++", "Rust", "Assembly", "Java", "Go"], label="Select Target")
        machine = gr.Textbox(label="Machine", interactive=False)
    with gr.Row(elem_classes=["black", "black-glow-card"]):
        convert = gr.Button("Convert code")
        # Automatically populate machine_info on load
        ui.load(get_machine_info, inputs=[], outputs=[machine])
    
    # Add execution buttons
    with gr.Row(elem_classes=["black", "black-glow-card"]):
        source_run = gr.Button("Run Source Code")
        target_run = gr.Button("Run Target Code")
    
    # Add output areas for execution results
    with gr.Row(elem_classes=["blur", "glass-card"]):
        source_output = gr.TextArea(label="Source Output:", lines=5, elem_classes=["blur", "glass-card"])
        target_output = gr.TextArea(label="Target Output:", lines=5, elem_classes=["blur", "glass-card"])
    
    # Connect the buttons to functions
    convert.click(optimize, inputs=[model, source_lang, source_code, machine, target_lang,], outputs=[target_code])
    source_run.click(execute_source_code, inputs=[source_lang, source_code], outputs=[source_output])
    target_run.click(execute_target_code, inputs=[target_lang, target_code], outputs=[target_output])

if __name__ == "__main__":
    ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
